In [2]:
!pip install --upgrade google-cloud-language

In [27]:
from google.cloud import language_v1

In [28]:
import pandas as pd

In [29]:
client = language_v1.LanguageServiceClient()

In [102]:
sample_docs = pd.read_csv("sample.csv")

In [149]:
sample_docs["text"][9]

'I think it is one of the best games , I have ever played. There can be some additions. For example. There should be a feature which tells about the stats of that player with my team. There should be representation when there is substitution. There should be more tournaments except the casual matches which could provide us with trophies and boots. These features could make the game more interactive and addictive'

In [34]:
type_ = language_v1.Document.Type.PLAIN_TEXT
language = "ja"
document = {"content": sample_text, "type_":type_, "language": language}
encoding_type = language_v1.EncodingType.UTF8

In [35]:
document

{'content': 'アプデ前まで普通に出来てたのが、アプデ後になっていきなりプレーが切れる度に重くなったりフリーズするようになった。できる限りの事はやったけど変わらなかった。これで回線切れることもたまにあるのでどうにかして欲しい。',
 'type_': <Type.PLAIN_TEXT: 1>,
 'language': 'ja'}

# API関連部分を関数化

In [76]:
def analyze_sentiment(document):
    responce = client.analyze_sentiment(
        request= {'document': document}
    )
    
    li = []
    for i in responce.sentences:
        li.append(
            [
                i.text.content,
                i.sentiment.score
            ]
        )
    return li

In [92]:
exclude_type_ = []
def analyze_entity(document):
    responce = client.analyze_entities(
        request= {'document': document,}
    )
    li = []
    for i in responce.entities:
        li.append([i.name, i.type_])
        
    return li

In [120]:
def analyze_documents(df):
    li = []
    for i in sample_docs.values:
        if i[1] == "ja":
            language = "ja"
        else:
            language = "en"
        type_ = language_v1.Document.Type.PLAIN_TEXT
        document = {"content": i[2], "type_":type_, "language": language}
        
        sentiment_res = analyze_sentiment(document)
        entitiy_res = analyze_entity(document)
        index = i[0]
        li.append([index, sentiment_res, entitiy_res])
    return li

# 分析実行 / 加工

In [122]:
result = analyze_documents(sample_docs)

In [128]:
# sentiment用にデータ加工
li = []
for i in result:
    index_ = i[0]
    
    for j in i[1]:
        li.append([index_, j[0], j[1]])

In [134]:
sentiment_result = pd.DataFrame(li, columns = ["index", "content", "sentiment"])
sentiment_result.to_csv("sentiment_result.csv")

In [145]:
# entity用にデータ加工
li = []
for i in result:
    index_ = i[0]
    
    for j in i[2]:
        li.append([index_, j[0], j[1].name])

In [146]:
entity_result = pd.DataFrame(li, columns = ["index", "entitiy", "type"])
entity_result.to_csv("entity_result.csv")